### rev. 인베스팅 실시간 선물 데이터 파일다운로드 없이 불러오기
- 받아온 정보를 데이터 프레임으로 만들어서 csv 로 저장하는 구조.
- 이전 코드는 장이 돌아가는 실시간 상황에서는 유효하지 않았다.(class 명이 실시간으로 바뀌기 때문)
  => 코드 수정을 통해 해결했고,변동값(change)까지 추가하는 코드로 수정했다. 실시간상황에도 돌아간다.
 
- 이전코드는 클래스 네임(pid-xxx-xx) 를 받아온 이후에, 그 이름을 이용해서 다시 driver로 Webelement를 선택하는 방법이어서 실시간 상황에서 부적합했다 => 처음 선택한 element 를 계속 유지하는 것으로 코드 변경(실시간에도 문제없음)
 

- 실제 쓸때는 데이터프레임 구조는 손봐야 할거 같다.

- 반복실행시 csv 파일 존재하면, 에러난다. 덮어쓰면서 업데이트 하도록 추가하기
=> 차라리 데이터 프레임 구조를 만들어 놓고, 해당 데이터 프레임을 업데이트한 후 저장하는 게 나을듯.
사실 일별로 매일 실행하는 것은 큰 의미는 없을듯. =>어차피 일별데이터는 나중에 한번에 받을 수도 있으니까.

- 인베스팅 시간 기준: 미국 워싱턴 DC


In [1]:
# 인베스팅 실시간 선물 데이터 페이지 14개
# 변동률까지 포함하도록 수정한 코드
# 실시간 환경에서 구동가능

import numpy as np
import pandas as pd

from selenium import webdriver
import time

driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)
driver.get('https://www.investing.com/commodities/real-time-futures')
# 선물 항목명 얻기(names)
elements = driver.find_elements_by_tag_name('a')
names = [i.text for i in elements[315:351]]  # 이름 얻어오는 슬라이싱 구간이 변할수도 있을 것 같다. 다른 날 다시 테스트해 볼것.

# 종가, high, low, time 얻기
# 처음부터 pid 객체로 얻기.<td class="pid-8894-high">2,662.00</td>
elements = driver.find_elements_by_css_selector("td[class*='pid']")
numbers = [i.text for i in elements]
numbers = numbers[:216]   # 필요한 선물데이터 영역만 슬라이싱

price_last=[numbers[6*i] for i in range(36)]
price_high=[numbers[6*i+1] for i in range(36)]
price_low=[numbers[6*i+2] for i in range(36)]
price_pc=[numbers[6*i+3] for i in range(36)]
price_pcp=[numbers[6*i+4] for i in range(36)]
price_time=[numbers[6*i+5] for i in range(36)]

df = pd.DataFrame({
    'Time':price_time
    , 'Commodity': names
    , 'Last':price_last
    , 'High':price_high
    , 'Low':price_low
    , 'Change':price_pc
    , 'Change(%)':price_pcp
})

# csv 저장위치&파일명 기입
df.to_csv('./data/commodity_real_time_test12.csv'
         , sep=','
          , encoding='utf-8'
         )

driver.close()

pid 부분 수정해서 change 부분까지 가져오자

In [80]:
# 처음부터 pid 객체로 얻기.

driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)
driver.get('https://www.investing.com/commodities/real-time-futures')

# 종가, high, low, time 얻기

elements = driver.find_elements_by_css_selector("td[class*='pid']")

numbers = [i.text for i in elements]
numbers = numbers[:216]

price_last=[numbers[6*i] for i in range(36)]
price_high=[numbers[6*i+1] for i in range(36)]
price_low=[numbers[6*i+2] for i in range(36)]
price_pc=[numbers[6*i+3] for i in range(36)]
price_pcp=[numbers[6*i+4] for i in range(36)]
price_time=[numbers[6*i+5] for i in range(36)]

df = pd.DataFrame({
    'Time':price_time
    , 'Commodity': names
    , 'Last':price_last
    , 'High':price_high
    , 'Low':price_low
    , 'Change':price_pc
    , 'Change(%)':price_pcp
})

df.to_csv('./data/commodity_real_time_test33333.csv'
         , sep=','
          , encoding='utf-8'
         )

driver.close()


프로세스가 완료될때까지 driver 는 열려있는 것이 안전하다. 닫은 후에는 이미 선택되었다 해도 element를 인식 못하는 것 같다

In [94]:
driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)
driver.get('https://www.investing.com/commodities/real-time-futures')

In [95]:
elements = driver.find_elements_by_css_selector("td[class*='pid']")

In [96]:
elements2 = driver.find_elements_by_css_selector("td[class*='pid']")

In [92]:
elements[0].text

InvalidSessionIdException: Message: invalid session id


In [98]:
numbers = [i.text for i in elements2]
numbers = numbers[:216]

price_last=[numbers[6*i] for i in range(36)]
price_high=[numbers[6*i+1] for i in range(36)]
price_low=[numbers[6*i+2] for i in range(36)]
price_pc=[numbers[6*i+3] for i in range(36)]
price_pcp=[numbers[6*i+4] for i in range(36)]
price_time=[numbers[6*i+5] for i in range(36)]

df = pd.DataFrame({
    'Time':price_time
    , 'Commodity': names
    , 'Last':price_last
    , 'High':price_high
    , 'Low':price_low
    , 'Change':price_pc
    , 'Change(%)':price_pcp
})

df.to_csv('./data/commodity_real_time_test_3rd.csv'
         , sep=','
          , encoding='utf-8'
         )


# 인베스팅 자동로그인해서 historical data 다운받기

stackoverflow 다른 유저들이 시도하는 코드들

In [ ]:
from selenium import webdriver
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", './ogdc.csv')
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

driver = webdriver.Firefox(firefox_profile=profile)
driver.get('https://www.investing.com/equities/oil---gas-dev-historical-data')
driver.find_element_by_class_name("login bold")
driver.find_element_by_id('Email').send_keys('myemail')
driver.find_element_by_id('Password').send_keys('mypass')
driver.find_element_by_partial_link_text("Download Data").click()

In [ ]:
driver.get('https://www.investing.com/equities/oil---gas-dev-historical-data')
driver.find_element_by_css_selector("a[class*='login']").click()
driver.find_element_by_id('loginFormUser_email').send_keys('myemail')
driver.find_element_by_id('loginForm_password').send_keys('mypass')
driver.find_element_by_xpath("//div[@id='loginEmailSigning']//following-sibling::a[@class='newButton orange']").click()

### 인베스팅 따로 계정을 만들어서 로그인한다. (구글, 페북 연계 X)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)
driver.get('https://www.investing.com/commodities/gold-historical-data')


In [ ]:
driver.get('https://www.investing.com/commodities/gold-historical-data')

In [ ]:
#과거 데이터 다운로드 버튼 클릭하기
# 과거데이터 다운로드 클릭 버튼 xpath
xpath = '''//*[@id="column-content"]/div[4]/div/a'''

# 조회 xpath 단일값 선택
element_sel_gu = driver.find_element_by_xpath(xpath).click()

### 1단계 완성코드 :
-  특정 1가지 종목 1month historical data 획득 : 간단한 기본 구현

In [ ]:
# 특정 종목 1month historical data 획득 : 1단계 간단한 구현
from selenium import webdriver
import time

driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)
# 인베스팅 선물 '금' 항목 페이지 접속
driver.get('https://www.investing.com/commodities/gold-historical-data')
# 로그인하기
driver.find_element_by_css_selector("a[class*='login']").click()
time.sleep(1)
driver.find_element_by_id('loginFormUser_email').clear()
driver.find_element_by_id('loginFormUser_email').send_keys('idmailaddress')
time.sleep(0.5)
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginForm_password').send_keys('password')
time.sleep(0.8)
driver.find_element_by_xpath("//div[@id='loginEmailSigning']//following-sibling::a[@class='newButton orange']").click()
#driver.find_element_by_xpath('''//*[@id="signup"]/a''').click()  # 얘로 하면 안될때도 있다.
time.sleep(1.2)
#과거 데이터 다운로드 버튼 클릭하기
driver.find_element_by_xpath('''//*[@id="column-content"]/div[4]/div/a''').click()
time.sleep(1.1)

driver.close()

## 반복문으로 모든 commodities 파일 다운받기
- url 반복문 코드 작성하기

### 1차 완성코드
- 브라우저로 과정이 보이니 편하다.
- 생각보다 충돌이 없이 다운로드 된다.(장중에 오류없는지 다시 코드 실행해볼것)
- 선물 뿐만아니라 주소값 조정해서 다른 항목도 가져올 수 있다.
- 예외상황이 있을수 있으니 try except 추가할것.

#### 아래 마이너한 수정하기
- 자동로그인 메서드로 만들기

In [3]:
import numpy as np
import pandas as pd

from selenium import webdriver
import time

# 반복문으로 14개의 선물 과거데이터 다운로드 받기

driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)
# 인베스팅 선물 항목 페이지 접속
driver.get('https://www.investing.com/commodities/')

# 로그인하기
driver.find_element_by_css_selector("a[class*='login']").click()
time.sleep(1)
driver.find_element_by_id('loginFormUser_email').clear()
driver.find_element_by_id('loginFormUser_email').send_keys('gkdkssk21@naver.com')
time.sleep(0.5)
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginForm_password').send_keys('wndml12')
time.sleep(0.8)
driver.find_element_by_xpath("//div[@id='loginEmailSigning']//following-sibling::a[@class='newButton orange']").click()
time.sleep(1)

# 반복문으로 활용하기 위한 url 리스트를 얻기 위해 href attribute를 get해서, url 을 얻기
tds = driver.find_elements_by_css_selector("td")
    
a =[]
for td in tds:
    try:
        a.append(td.find_element_by_css_selector("a"))
    except:
        pass

# href 값을 얻어 url 리스트를 획득한다.
urls = [item.get_attribute('href') for item in a]
urls = list(set(urls))

# commodities 의 주소 리스트를 얻기.
commodities_url = []
for url in urls:
    if 'commodities' in url:
        commodities_url.append(url)

# 반복문으로 14개의 선물 과거데이터 다운로드 받기
for url in commodities_url:
    tmp_url = url + '-historical-data'
    driver.get(tmp_url)
    time.sleep(1.1)
    #과거 데이터 다운로드 버튼 클릭하기
    driver.find_element_by_xpath('''//*[@id="column-content"]/div[4]/div/a''').click()
    time.sleep(0.6)

driver.close()
    

### 추후 과제 
- 다운로드 받을 날짜 지정하는 코드 작성
- 기존 파일 혹은 데이터프레임에 데이터 업데이트하는 코드 작성

아래부터는 완성 코드 작성 과정

In [5]:
from selenium import webdriver
import time

In [ ]:
tmp_url

'https://www.investing.com/commodities/gold-historical-data'

In [7]:
driver = webdriver.Chrome('./webdriver/chromedriver')
time.sleep(1)

driver.get('https://www.investing.com/commodities/')

In [8]:
tds = driver.find_elements_by_css_selector("td")

In [9]:
tds

[<selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="5a356236-19ac-4d8b-9b95-81862ac5f560")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="1e74b1e2-58c7-47ec-995a-7dfba0c0f6d7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="527b21e9-4415-4f0f-9528-3687022fb29d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="3e4db288-63a1-4a5a-9a9f-f2da56cbebad")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="f4388df2-c81a-44ee-a5ed-3546502efe6f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="207bb965-eb1b-4404-b77e-f126e980842e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="98691e18-601c-4d22-b288-a8

In [22]:
a =[]
for td in tds:
    try:
        a.append(td.find_element_by_css_selector("a"))
    except:
        pass

In [23]:
a

[<selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="04322d5c-298b-42ed-84dd-cf50eebb7051")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="2473028e-e15b-428a-83c8-bc8dad12703f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="31657389-2947-4b0f-9cf6-69a6bc583241")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="574b5a87-4324-4329-8d32-33f761b7ef5a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="3035f717-941e-4aae-af70-9d3272abcb17")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="9fcf80e2-c80f-445c-bf3d-70e898ca4d2d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9e3da16ae5303abbb1c2862e90c9a9ab", element="9344ce75-960c-401a-a8ed-e1

In [24]:
urls = [item.get_attribute('href') for item in a]
urls = list(set(urls))

['https://www.investing.com/equities/facebook-inc',
 'https://www.investing.com/currencies/eur-usd',
 'https://www.investing.com/commodities/london-sugar',
 'https://www.investing.com/commodities/us-cotton-no.2',
 'https://www.investing.com/indices/reuters-jefferies-crb-natural-gas',
 'https://www.investing.com/indices/us-spx-500',
 'https://www.investing.com/currencies/eur-chf',
 'https://www.investing.com/indices/dj-commodity',
 'https://www.investing.com/etfs/ishares-silver-trust',
 'https://www.investing.com/currencies/aud-usd',
 'https://www.investing.com/equities/apple-computer-inc',
 'https://www.investing.com/equities/google-inc',
 'https://www.investing.com/indices/reuters-jefferies-crb-gold',
 'https://www.investing.com/etfs/proshares-trust-ultrapro-qqq',
 'https://www.investing.com/commodities/copper',
 'https://www.investing.com/etfs/spdr-s-p-500',
 'https://www.investing.com/equities/amazon-com-inc',
 'https://www.investing.com/indices/nasdaq-composite',
 'https://www.inve

In [ ]:
# 반복문으로 활용하기 위한 url 리스트를 얻기 위해 href attribute를 get해서, url 을 얻는다.

# <td class="noWrap bold left noWrap">
#   <a title="Platinum Futures (CFD)" href="/commodities/platinum">Platinum</a>
#  <span class="alertBellGrayPlus js-plus-icon genToolTip oneliner" data-tooltip="Create Alert" data-name="Platinum Futures" data-id="8910"></span></td>

tds = driver.find_elements_by_css_selector("td")
a =[]
for td in tds:
    if td.find_elements_by_css_selector("a")!=[]:
        a.append(td.find_elements_by_css_selector("a"))
    else:pass
# href 값을 얻어 url 을 획득한다.
urls = [item[0].get_attribute('href') for item in a]
urls = list(set(urls))

In [ ]:
# 이 코드에서는 by_tag_name으로 선택해도 결과는 같다. 무슨 차이일까?
tds = driver.find_elements_by_tag_name("td")
a =[]
for td in tds:
    if td.find_elements_by_tag_name("a")!=[]:
        a.append(td.find_elements_by_css_selector("a"))
    else:pass
# href 값을 얻어 url 을 획득한다.
urls = [item[0].get_attribute('href') for item in a]
urls = list(set(urls))

In [ ]:
len(urls)

47

In [ ]:
# url의 sub 부분을 기준으로 해서 카테고리별로 리스트로 나누기
commodities_url = []
etfs_url= []
currencies_url = []
indices_url=[]
equities_url=[]

for url in urls:
    if 'commodities' in url:
        commodities_url.append(url)
    elif 'etfs' in url:
        etfs_url.append(url)
    elif 'currencies' in url:
        currencies_url.append(url)
    elif 'indices' in url:
        indices_url.append(url)
    elif 'equities' in url:
        equities_url.append(url)
        

In [ ]:
# commodities 의 주소 리스트를 얻었다.
commodities_url

['https://www.investing.com/commodities/heating-oil',
 'https://www.investing.com/commodities/gold',
 'https://www.investing.com/commodities/copper',
 'https://www.investing.com/commodities/us-cotton-no.2',
 'https://www.investing.com/commodities/brent-oil',
 'https://www.investing.com/commodities/natural-gas',
 'https://www.investing.com/commodities/us-coffee-c',
 'https://www.investing.com/commodities/us-soybeans',
 'https://www.investing.com/commodities/us-wheat',
 'https://www.investing.com/commodities/us-corn',
 'https://www.investing.com/commodities/crude-oil',
 'https://www.investing.com/commodities/platinum',
 'https://www.investing.com/commodities/silver',
 'https://www.investing.com/commodities/london-sugar']

### 번외1.  BeautifulSoup 시도 -> 실패
- 인베스팅은 urlopen 막아놓은듯

In [ ]:
# url 이용해서 html 파일 가져오기 => 오류난다. 인베스팅은 urlopen 막아놓은듯하다.
from bs4 import BeautifulSoup
from urllib.request import urlopen     #url : 웹상 주소, open : 파일 처리
url_base = 'https://www.investing.com'
url_sub = '/commodities/gold-historical-data'
url = url_base + url_sub
html = urlopen(url)
# html 파싱
soup = BeautifulSoup(html, 'html.parser')

HTTPError: HTTP Error 403: Forbidden

### 번외2(네이버크롤링). web 모듈(파이참 모듈 참고 by hccho) 작동 test 
- 네이버 일별 시세 페이지를 파싱. 주식(종가, 시가, high, row, 거래량) 가져오는 것이 목적
- beautifulsoup 을사용,  모든 종목에 대해, 각 종목 당 하나의 csv 파일을 생성하고 매일 업데이트 되도록 만든 모듈.
- csv 로 저장하면서 동시에 db에도 넣는 메소드가 서로 유기적으로 연결되어 있는것이, 데이터 정확도 측면에서 좋을 것 같긴하지만 시스템적으로 좋아보이지 않는다.
- 만약 시스템에 구멍이 나면 뭐가 에러났는지 직관적으로 알기 힘들고, 프로세스 자체가 무겁게 느껴진다.
- 메소드는 가능한 분리시키되, 데이터 정확성은 유지하는 방법을 찾아야 겠다.
- 한번에 하나씩 순차적으로 되게 하는게 좋은 거 같다.
- 해당 모듈은 크롤링과 db 연동 연습에 좋은 것 같다
- 실상황에서는, 분석 종목을 줄이는게 시스템에 안정적이겠다. 종목이 너무 많다, 쓸데없는 것들은 빼는게 오히려 분석에 좋을 거 같다.

In [ ]:
import pandas as pd

In [ ]:
df =pd.read_csv('./data/code.csv'
               , index_col=0
               )

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 6-7: invalid continuation byte

In [ ]:
df.to_csv('./data/code.csv')

In [ ]:
df2 = df.set_index('종목코드')

In [ ]:
df2

,종목명
종목코드,
000020,동화약품
000040,KR모터스
000050,경방
000060,메리츠화재
000070,삼양홀딩스
...,...
590018,미래에셋 중국 심천 100 ETN
590012,미래에셋 글로벌 리츠 ETN(H)
700001,하나 코스피 변동성추세 추종 양매도 ETN


In [ ]:
df2.to_csv('./data/code2.csv')

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [ ]:
def data_list(web_url):
    with urlopen(web_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        span_list = soup.find_all('span')
        del span_list[0]
        result_list = []
        for i in range(len(span_list)):
            data = str(span_list[i].get_text()).strip()
            data_re = re.sub('[\.\,]+', "", data)  # 정규식 [,]또는 [.] 없애기
            result_list.append(data_re)
    return result_list

In [ ]:
a =data_list('https://finance.naver.com/item/sise_day.nhn?code=006490&page=1')

In [ ]:
len(a)

70

In [ ]:
int(len(a) / 7)+1

11